## We want to build a model of demand and supply of publicly provided services in a city

In [7]:
import geopy
import numpy as np

In [10]:
from enum import Enum

class ageGroups(Enum):
    Newborn= (0, 3)
    Child= (4,15)
    Young= (16,25)
    Junior= (26,35)
    Senior= (36, 50)
    Over50= (50, 64)
    Over65= (66, 80)
    Over80= (81, 200)
    def __init__(self, startAge, endAge):
        self.start = startAge
        self.end = endAge
    @staticmethod
    def all():
        return([g for g in ageGroups])
    @property
    def range(self): return self.end - self.start
    
    
class Services(Enum):
    School = 1
    Football = 2
    Doctor = 3
    #etc
    
kPropagationTest = 3

class ServiceUnit:
    def __init__(self, name='', position=(45.4641, 9.1919), users=ageGroups.all(), times=[], scale=1):
        assert isinstance(position, tuple) & (len(position) == 2), 'Position must be a pair tuple' #will be replaced by nicer class
        self.name = name
        self.site = position  # a Service has many sites, and a site is not uniquely assigned to a service
        self.users = users
        self.times = times
        self.propagation ={g: (1 + .1*np.random.normal())*kPropagationTest*scale for g in self.users} 
        # how the service is available for different age groups
        
        



In [11]:
test = ServiceUnit('Duomo')
test.propagation

{<ageGroups.Child: (4, 15)>: 2.7984163700339124,
 <ageGroups.Junior: (26, 35)>: 3.236271276089605,
 <ageGroups.Newborn: (0, 3)>: 3.3393112390801205,
 <ageGroups.Over50: (50, 64)>: 3.365449703165874,
 <ageGroups.Over65: (66, 80)>: 3.2160549678709485,
 <ageGroups.Over80: (81, 200)>: 3.082434115248339,
 <ageGroups.Senior: (36, 50)>: 2.965063886635803,
 <ageGroups.Young: (16, 25)>: 2.75652041272508}

In [4]:
### Supply modeling
